<a href="https://colab.research.google.com/github/manuel-suarez/introduction_to_gans/blob/main/Generative_Adversarial_Networks_in_Slanted_Land.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports

In [1]:
# Imports

import numpy as np
from numpy import random
from matplotlib import pyplot as plt
%matplotlib inline